In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/att532tsp/att532.tsp
/kaggle/input/d1291-tsplib-dataset/d1291.tsp


In [45]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from multiprocessing import Pool #Multithreading
import time 
import random


#Import "att532.tsp" file to read the cities (http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/)
def read_cities(filename):
    with open(filename, 'r') as f:
        # Skip the header lines
        for i in range(6):
            next(f)
        
        # Extract the coordinates and build the tuples
        cities = []
        for line in f:
            if line.startswith('EOF'):
                break
            values = line.split()
            if len(values) >= 3:
                x, y = values[1], values[2]
                cities.append((float(x), float(y)))
    return cities

In [46]:
def ant_movement(ant, cities, pheromone, heuristic, alpha, beta, q0):
    for j in range(len(cities)-1):
        current_city = ant[-1]
        unvisited_cities = [city for city in range(len(cities)) if city not in ant]
        if not unvisited_cities:
            break
        probabilities = []
        for city in unvisited_cities:
            pheromone_level = pheromone[current_city][city]
            heuristic_info = heuristic[current_city][city]
            probability = (pheromone_level**alpha) * (heuristic_info**beta)
            probabilities.append(probability)
        if not probabilities:
            next_city = ant[0]
        elif random.random() < q0:
            next_city = unvisited_cities[np.argmax(probabilities)]
        else:
            total_prob = sum(probabilities)
            probabilities = [p/total_prob for p in probabilities]
            next_city = np.random.choice(unvisited_cities, p=probabilities)
        ant.append(next_city)
    return ant

def ant_colony_stp_parallel(cities, start_city, num_ants, alpha, beta, q0, pheromone_decay, num_iterations, num_processes):
    # Step 2: Initialize the ants
    ants = [[start_city] for i in range(num_ants)]

    # Step 3: Define the pheromone matrix
    pheromone = np.ones((len(cities), len(cities))) * 0.1

    # Step 4: Define the heuristic information matrix
    heuristic = np.zeros((len(cities), len(cities)))
    for i in range(len(cities)):
        for j in range(len(cities)):
            heuristic[i][j] = np.sqrt((cities[i][0]-cities[j][0])**2 + (cities[i][1]-cities[j][1])**2)

    # Step 5: Define ant's movement rules
    with Pool(num_processes) as pool:
        for iteration in range(num_iterations):
            # Parallelize ant movement
            ants_args = [(ant, cities, pheromone, heuristic, alpha, beta, q0) for ant in ants]
            ants = pool.starmap(ant_movement, ants_args)

            # Update the pheromone levels
            for i in range(num_ants):
                path_length = sum([heuristic[ants[i][j]][ants[i][j+1]] for j in range(len(cities)-1)])
                for j in range(len(cities)-1):
                    pheromone[ants[i][j]][ants[i][j+1]] += 1/path_length
                pheromone *= (1-pheromone_decay)

    # Once you have converged, you can return the best path found by the ants.
    best_path_indices = ants[np.argmin([sum([heuristic[ants[i][j]][ants[i][j+1]] for j in range(len(cities)-1)]) for i in range(num_ants)])]
    best_path = [cities[i] for i in best_path_indices]
    total_distance = sum([heuristic[best_path_indices[j]][best_path_indices[j+1]] for j in range(len(cities)-1)])
    return best_path, total_distance

In [47]:
#Defining method parameters
cities = read_cities('/kaggle/input/att532tsp/att532.tsp')
start_city = 0
num_ants = 10
alpha = 1
beta = 2
q0 = 0.9
pheromone_decay = 0.1
num_iterations = 100
num_processes = 2 # For the parallel solution
start_time = time.time_ns() // 1000 
best_path, total_distance = ant_colony_stp_parallel(cities, start_city, num_ants, alpha, beta, q0, pheromone_decay, num_iterations, num_processes)
end_time = time.time_ns() // 1000 
elapsed_time = end_time - start_time
print("WITH PARALLELISM, 532 CITIES AND {} THREADS ".format(num_processes))
print("Elapsed time: {} microseconds".format(elapsed_time))
#print("Best path:", best_path)
print("Total distance:", total_distance)

WITH PARALLELISM, 532 CITIES AND 2 THREADS 
Elapsed time: 16626814 microseconds
Total distance: 2157604.83462736
